In [1]:
%reset -f

In [1]:
%precision 2
import numpy as np

class Leaky_ReLU:
    @staticmethod
    def forward(x):
        data = [[max(0.05*value,value) for value in row] for row in x]
        return np.array(data, dtype=float)
    @staticmethod
    def backward(x):
        data = [[1 if value>0 else 0.05 for value in row] for row in x]
        return np.array(data, dtype=float)

class ReLU:
    @staticmethod
    def forward(x):
        return np.maximum(0, x)
    @staticmethod
    def backward(x):
        return 1. * (x > 0)
    
class tanh:
    @staticmethod
    def forward(x):
        return np.tanh(x)
    @staticmethod
    def backward(x):
        return 1-np.tanh(x)**2

class sigmoid:
    @staticmethod
    def forward(x):
        return 1/(1+np.exp(-x))
    @staticmethod
    def backward(x):
        fx = sigmoid.forward(x)
        return fx*(1-fx)

# loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_pred-y_true, 2))
def mse_prime(y_true, y_pred):
    return 2*np.mean(y_pred-y_true)

# Base class
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
    def forward_propagation(self, input):
        raise NotImplementedError
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

class ActivationLayer(Layer):
    def __init__(self, function):
        self.activation = function.forward
        self.activation_prime = function.backward
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

class FCLayer(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size)
        self.bias = np.random.rand(1, output_size)
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

class Network:
    def __init__(self, loss, loss_prime):
        self.layers = []
        self.loss = loss
        self.loss_prime = loss_prime
    def add(self, layer):
        self.layers.append(layer)
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime
    def predict(self, input_data):
        sample_count = input_data.shape[0]
        result = []
        for i in range(sample_count):
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)
        return result
    def fit(self, x_train, y_train, epochs, learning_rate):
        x_train = np.array([[elem] for elem in x_train])
        y_train = np.array([[elem] for elem in y_train])
        err = 0
        for i in range(epochs):
            err = 0
            for j in range(x_train.shape[0]):
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)
                err += self.loss(y_train[j], output)
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)
        print(f'fit result of {epochs} epochs, error={err}, learning_rate={learning_rate}')
        return err

In [2]:
def normalize_data(data, prefix=''):
    minimum = np.min(data)
    maximum = np.max(data)
    diff = maximum - minimum
    bias = minimum if minimum >= 0 else -minimum
    normalized_data = (data-bias)/diff
    print("{prefix}diff={diff}, {prefix}bias={bias}")
    return normalized_data, diff, bias

def training_compare(x_train, y_train, output, x_diff=0, x_bias=0, y_diff=0, y_bias=0):
    if x_diff==0 and x_bias==0:
        display_x_train = x_train
    else:
        display_x_train = np.copy(x_train)*x_diff+x_bias
    if y_diff==0 and y_bias==0:
        display_y_train = y_train
        display_output = output
    else:
        display_y_train = np.copy(y_train)*y_diff+y_bias
        display_output = np.copy(output)*y_diff+y_bias
    for i in range(display_x_train.shape[0]):
        print(f'expected {display_x_train[i]}\tto {display_y_train[i]}\tgot {display_output[i]}')

def prediction_display(input, output, x_diff=0, x_bias=0, y_diff=0, y_bias=0):
    if x_diff==0 and x_bias==0:
        display_input = input
    else:
        display_input = np.copy(input)*x_diff+x_bias
    if y_diff==0 and y_bias==0:
        display_output = output
    else:
        display_output = np.copy(output)*y_diff+y_bias
    for i in range(display_input.shape[0]):
        print(f'inputed { display_input[i]}\tgot {display_output[i]}')

In [3]:
x_data = np.array([[85],[88],[91],[94],[97],[100],[103],[106],[109],[112],[115]])
y_data = np.array([[28800],[14400],[7200],[3600],[1800],[900],[450],[225],[112.5],[56.25],[28.125]])

x_train, x_diff, x_bias = normalize_data(x_data, 'x_')
y_train, y_diff, y_bias = normalize_data(y_data, 'y_')

net = Network(mse, mse_prime)
net.add(FCLayer(x_data.shape[1], 5))
net.add(ActivationLayer(tanh))
net.add(FCLayer(5, 4))
net.add(ActivationLayer(tanh))
net.add(FCLayer(4, y_data.shape[1]))
lr = 0.1
for i in range(5):
    net.fit(x_train, y_train, epochs=100, learning_rate=lr)
    lr /= 1.25

out = net.predict(x_train)
training_compare(x_train, y_train, out, x_diff, x_bias, y_diff, y_bias)

{prefix}diff={diff}, {prefix}bias={bias}
{prefix}diff={diff}, {prefix}bias={bias}
fit result of 100 epochs, error=0.06423651826423267, learning_rate=0.1
fit result of 100 epochs, error=0.00661028648235658, learning_rate=0.08
fit result of 100 epochs, error=0.002575767501378715, learning_rate=0.064
fit result of 100 epochs, error=0.0015051570722327632, learning_rate=0.0512
fit result of 100 epochs, error=0.0010511401627237979, learning_rate=0.04096
expected [85.]	to [28800.]	got [[28432.0962456]]
expected [88.]	to [14400.]	got [[14965.67130616]]
expected [91.]	to [7200.]	got [[7051.79013725]]
expected [94.]	to [3600.]	got [[3322.03534113]]
expected [97.]	to [1800.]	got [[1645.43925394]]
expected [100.]	to [900.]	got [[861.56938852]]
expected [103.]	to [450.]	got [[469.75010018]]
expected [106.]	to [225.]	got [[259.98880383]]
expected [109.]	to [112.5]	got [[140.49331268]]
expected [112.]	to [56.25]	got [[68.6105767]]
expected [115.]	to [28.125]	got [[23.25249825]]


In [4]:
new_samples = np.array([[(x-x_bias)/x_diff] for x in np.linspace(80, 120, 10)])
out = net.predict(new_samples)
prediction_display(new_samples, out, x_diff, x_bias, y_diff, y_bias)

inputed [80.]	got [[51411.81198276]]
inputed [84.44444444]	got [[31322.66311365]]
inputed [88.88888889]	got [[12042.16691598]]
inputed [93.33333333]	got [[3913.2576634]]
inputed [97.77777778]	got [[1384.46150349]]
inputed [102.22222222]	got [[548.30177942]]
inputed [106.66666667]	got [[227.63330411]]
inputed [111.11111111]	got [[86.45227045]]
inputed [115.55555556]	got [[16.78204563]]
inputed [120.]	got [[-21.03067827]]


In [5]:
x_data = np.array([[0, 0, 1],[0, 1, 1],[1, 0, 1],[1, 1, 1]])
y_data = np.array([[0],[1],[1],[0]])

x_train, x_diff, x_bias = normalize_data(x_data, 'x_')
y_train, y_diff, y_bias = normalize_data(y_data, 'y_')

net = Network(mse, mse_prime)
net.add(FCLayer(x_data.shape[1], 5))
net.add(ActivationLayer(tanh))
net.add(FCLayer(5, 4))
net.add(ActivationLayer(tanh))
net.add(FCLayer(4, y_data.shape[1]))
lr = 0.1
for i in range(5):
    net.fit(x_train, y_train, epochs=100, learning_rate=lr)
    lr /= 1.25

out = net.predict(x_train)
training_compare(x_train, y_train, out, x_diff, x_bias, y_diff, y_bias)

{prefix}diff={diff}, {prefix}bias={bias}
{prefix}diff={diff}, {prefix}bias={bias}
fit result of 100 epochs, error=1.9239013383131458, learning_rate=0.1
fit result of 100 epochs, error=0.0044540224088059945, learning_rate=0.08
fit result of 100 epochs, error=4.307005520250428e-11, learning_rate=0.064
fit result of 100 epochs, error=1.4139742371368468e-16, learning_rate=0.0512
fit result of 100 epochs, error=1.9078084133138852e-20, learning_rate=0.04096
expected [0. 0. 1.]	to [0.]	got [[1.0487633e-10]]
expected [0. 1. 1.]	to [1.]	got [[1.]]
expected [1. 0. 1.]	to [1.]	got [[1.]]
expected [1. 1. 1.]	to [0.]	got [[-3.13247206e-11]]
